In [1]:
import jax.numpy as jnp

import matplotlib.pyplot as plt
import tornadox
from matplotlib import animation

import pnmol

from IPython.display import HTML

In [2]:
%load_ext autoreload
%autoreload 2

## Discretize some 1D PDE

In [3]:
discretized_pde = pnmol.pde_problems.heat_1d()
# discretized_pde = pnmol.pde_problems.burgers_1d()

100%|███████████████████████████████████████████| 51/51 [00:01<00:00, 46.31it/s]

3.0 2.997601571978149
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.9999999740209646
3.0 2.999999

[[ 2496.00076557 -4995.0012311   2498.99996568 ...     0.
      0.             0.        ]
 [ 2500.50001752 -5000.99993505  2500.50001752 ...     0.
      0.             0.        ]
 [    0.          2500.50001752 -5000.99993505 ...     0.
      0.             0.        ]
 ...
 [    0.             0.             0.         ... -5000.99993505
   2500.50001752     0.        ]
 [    0.             0.             0.         ...  2500.50001752
  -5000.99993505  2500.50001752]
 [    0.             0.             0.         ...  2498.99996568
  -4995.0012311   2496.00076557]]
[[2.39842802e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.59790354e-08 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.59790354e-08 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.59790354e-08
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.0000

In [4]:
import jax
import jax.numpy as jnp

In [5]:
f = lambda x, y: jnp.exp(-(x-y)**2)

In [6]:
df = jax.grad(jax.grad(f, argnums=0), argnums=0)
df(2., 3.)

ddf = jax.grad(jax.grad(df, argnums=1), argnums=1)
ddf(2., 3.), df(2., 3.)

(DeviceArray(-7.35758882, dtype=float64),
 DeviceArray(0.73575888, dtype=float64))

In [7]:
D = pnmol.differential_operator.laplace()

ddf2 = D(D(f, argnums=0), argnums=1)

In [8]:
ddf2(jnp.array([2.]), jnp.array([3.]))

DeviceArray(-7.35758882, dtype=float64)

In [9]:
print(discretized_pde.E)

[[2.39842802e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.59790354e-08 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 2.59790354e-08 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.59790354e-08
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.59790354e-08 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 2.39842802e-03]]


In [10]:
assert False
constant_steps = tornadox.step.ConstantSteps(0.01)
adaptive_steps = tornadox.step.AdaptiveSteps(abstol=1e-3, reltol=1e-3)
nu = 3

AssertionError: 

In [ ]:
ek1 = pnmol.solver.MeasurementCovarianceEK0(num_derivatives=nu, steprule=adaptive_steps)
sol = ek1.solve(ivp=discretized_pde)

In [ ]:
print(sol.mean[3].shape)
print(sol.cov[3].shape)
E0 = ek1.iwp.projection_matrix(0)
print(E0.shape)

In [ ]:
plt.rcParams["animation.embed_limit"] = 2 * 10**8  # Set the animation max size to 200MB

grid = discretized_pde.spatial_grid

fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(1,1,1)
_im1 = ax.plot(grid.points.squeeze(), E0 @ sol.mean[0])
_im1 = ax.plot(grid.points.squeeze(), discretized_pde.y0.squeeze())

ax1ylim = [-0.2, 1.2]
ax.set_ylim(ax1ylim)

plt.close()


def animate(i):
    
    mean = E0 @ sol.mean[i]
    std = E0 @ jnp.sqrt(jnp.diag(sol.cov[i]))
    
    ax.cla()
    ax.set_title(f"t={sol.t[i]}")
    ax.plot(grid.points.squeeze(), mean, color="C0", label="PN solution")
    ax.fill_between(
        grid.points.squeeze(), 
        mean - 2 * std, 
        mean + 2 * std,
        color="C0",
        alpha=0.2,
    )
    ax.set_ylim(ax1ylim)
    ax.legend()
    
# Animation setup
anim = animation.FuncAnimation(
    fig, func=animate, frames=len(sol.t), interval=100, repeat_delay=4000, blit=False
)
HTML(anim.to_jshtml())